In [2]:
import pandas as pda
!pip install yfinance
import yfinance as yf



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\herbd\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
#this defines the asset you want to look at then fetches the data and displays it in a dataframe 
ticker_symbol = '^VIX'

vix_data = yf.download(ticker_symbol, start='2015-01-01', end='2024-12-04')

vix_data.columns = vix_data.columns.get_level_values(0)
vix_data.reset_index(inplace=True)

vix_data.columns = ['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

vix_data = vix_data[['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']]
vix_data.head(10)


[*********************100%***********************]  1 of 1 completed


,Date,Adj Close,Close,High,Low,Open,Volume
0,2015-01-02,17.790001,17.790001,20.139999,17.049999,17.760000,0
1,2015-01-05,19.920000,19.920000,21.290001,19.190001,19.190001,0
2,2015-01-06,21.120001,21.120001,22.900000,19.520000,20.330000,0
3,2015-01-07,19.309999,19.309999,20.719999,19.040001,20.150000,0
4,2015-01-08,17.010000,17.010000,18.090000,16.990000,17.930000,0
5,2015-01-09,17.549999,17.549999,18.420000,16.440001,16.440001,0
6,2015-01-12,19.600000,19.600000,20.440001,18.020000,18.020000,0
7,2015-01-13,20.559999,20.559999,21.580000,17.650000,18.209999,0
8,2015-01-14,21.480000,21.480000,23.340000,21.320000,22.870001,0
9,2015-01-15,22.389999,22.389999,23.309999,20.860001,21.230000,0


In [ ]:
#data for timerseries vix for layer 1
vix_close = vix_data[['Date', 'Close']]
vix_close.head(10)
vix_close.to_csv('vix_timeseries.csv', index=False)

Now i want data for layer 2 which is the spikes. I have already got code for this this in a different dataframe in a different folder but using different dates so can repeat it here, just for a differnt number of datapoints. 

In [16]:

# Filter to keep only the first instance where Close >= 30 and skip consecutive occurrences
filtered_vix_close = vix_close[(vix_close['Close'] >= 25) & 
                               (vix_close['Close'].shift(fill_value=0) < 25)]

filtered_vix_close.head(10)

#now i have the spike datapoints as a csv as well. this is for the second layer. 

,Date,Close
160,2015-08-21,28.030001
172,2015-09-09,26.230000
185,2015-09-28,27.629999
256,2016-01-08,27.010000
259,2016-01-13,25.219999
261,2016-01-15,27.020000
276,2016-02-08,26.000000
372,2016-06-24,25.760000
778,2018-02-05,37.320000
960,2018-10-24,25.230000


Now i want to download the whole bitcoin dataset that can then be filtered later in vegalite. 


In [14]:
#this defines the asset you want to look at then fetches the data and displays it in a dataframe 
ticker2 = 'BTC-USD'

btc_data = yf.download(ticker2, start='2015-01-01', end='2024-12-03')

# Flatten the headings so that its not across multiple levels
btc_data.columns = btc_data.columns.get_level_values(0)
print(btc_data.columns)

btc_data.reset_index(inplace=True)

btc_data.columns = ['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

btc_data = btc_data[['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']]

btc_close = btc_data[['Date', 'Close']]

btc_close['Date'] = pda.to_datetime(btc_close['Date']).dt.strftime('%Y-%m-%d')

btc_close.to_csv('BTC_dataset(1).csv', index=False) 


btc_close.head(10)


[*********************100%***********************]  1 of 1 completed

Index(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')



C:\Users\herbd\AppData\Local\Temp\ipykernel_21968\3394145315.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_close['Date'] = pda.to_datetime(btc_close['Date']).dt.strftime('%Y-%m-%d')


,Date,Close
0,2015-01-01,314.248993
1,2015-01-02,315.032013
2,2015-01-03,281.082001
3,2015-01-04,264.195007
4,2015-01-05,274.473999
5,2015-01-06,286.188995
6,2015-01-07,294.337006
7,2015-01-08,283.348999
8,2015-01-09,290.407990
9,2015-01-10,274.795990


In [21]:

btc_close['Date'] = pda.to_datetime(btc_close['Date'])

results = []

# loops thorugh all the VIX spike dates in filtered_vix_close dataframe
for index, row in filtered_vix_close.iterrows():
    vix_date = pda.to_datetime(row['Date']) 
    vix_date_label = vix_date.strftime('%Y-%m-%d')

#filters it for the 30days
    btc_subset = btc_close[(btc_close['Date'] > vix_date) & 
                           (btc_close['Date'] <= vix_date + pda.Timedelta(days=30))]
    
    # adds the filtered rows to the results with the VIX date label
    for _, btc_row in btc_subset.iterrows():
        results.append({
            'BTC_Date': btc_row['Date'],
            'BTC_Close': btc_row['Close'],
            'VIX_Date': vix_date_label
        })
#convets to a df
btc_vix_merged = pda.DataFrame(results)

btc_vix_merged.to_csv('BTCfiltere_data_intereactivechart.csv', index=False) 
btc_vix_merged.head(10)


C:\Users\herbd\AppData\Local\Temp\ipykernel_21968\3936131282.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_close['Date'] = pda.to_datetime(btc_close['Date'])


,BTC_Date,BTC_Close,VIX_Date
0,2015-08-22,230.389999,2015-08-21
1,2015-08-23,228.169006,2015-08-21
2,2015-08-24,210.494995,2015-08-21
3,2015-08-25,221.608994,2015-08-21
4,2015-08-26,225.830994,2015-08-21
5,2015-08-27,224.768997,2015-08-21
6,2015-08-28,231.395996,2015-08-21
7,2015-08-29,229.779999,2015-08-21
8,2015-08-30,228.761002,2015-08-21
9,2015-08-31,230.056000,2015-08-21
